# Imports

In [1]:
from pandas import Series
import pandas as pd
import pylab as pl
import numpy as np
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import cross_val_predict, StratifiedKFold, RandomizedSearchCV
from xgboost import XGBClassifier
from collections import Counter
import matplotlib.pyplot as plt
from scipy.stats import lognorm, uniform

/Users/mohan/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Dataset

In [2]:
df_test = pd.read_csv("test.csv")
df = pd.read_csv("train.csv")
df.head(20)

,Resident ID,Disease_Flag,var1,var2,var3,var4,var5,var6,var7,var8,...,var17,var18,var19,var20,var21,var22,var23,var24,var25,var26
0,A108705,0,6,34,-1,1096.633158,0.00,0,0,0,...,1.311183,1.552831,-3.000000,3.000000,92,5.000000,0,1,0,0
1,A100773,0,14,34,0,403.428793,0.03,0,0,0,...,1.296148,2.109440,1.457395,3.000000,91,3.461538,0,0,0,0
2,A102244,0,4,35,6,162754.791400,0.02,0,0,0,...,1.295839,1.833239,-3.000000,3.000000,94,3.214286,1,1,1,0
3,A102346,0,14,34,0,20.085537,0.04,0,0,0,...,1.295839,1.827844,-3.000000,3.000000,93,6.428571,0,1,0,1
4,A105178,0,9,34,0,59874.141720,0.80,0,0,0,...,1.295376,1.953927,1.454304,1.000000,93,4.090909,0,1,0,1
5,A111506,0,17,34,-1,442413.392000,0.02,0,0,0,...,1.264911,1.784367,1.435618,3.000000,92,5.000000,0,1,1,1
6,A110256,0,6,35,0,1.000000,0.30,0,0,0,...,1.264278,1.535313,1.381304,3.000000,92,5.000000,0,0,1,0
7,A100521,1,5,34,0,148.413159,0.10,0,0,0,...,1.131371,2.743765,1.697056,0.049192,95,4.090909,0,1,1,0
8,A104733,0,13,34,0,8103.083928,0.36,0,0,0,...,1.095445,2.390073,1.290349,0.078467,92,7.500000,0,0,1,0
9,A110448,0,7,34,0,22026.465790,0.05,0,0,0,...,1.095445,2.577059,1.604369,0.057126,92,4.090909,0,1,1,0


In [3]:
rid = df['Resident ID']
y = df['Disease_Flag'].values
X = df.iloc[:,2:].values
print(Counter(y))
print(y.shape)
print(X.shape)

Counter({0: 19400, 1: 600})
(20000,)
(20000, 26)


# Random search cross validation on xgboost model

In [4]:
clf = XGBClassifier(max_depth=4, learning_rate=0.1,
                    n_estimators=50, silent=True,
                    objective="binary:logistic",
                    nthread=1, gamma=0, min_child_weight=1,
                    max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=0.35,
                    reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                    base_score=0.5)
cv = StratifiedKFold(n_splits=5, shuffle=True)

In [ ]:
param_distributions = {"max_depth": range(2,10),
                      "learning_rate": uniform(loc=0, scale=0.2),
                      "n_estimators": range(10, 500),
                      "min_child_weight": range(1, 50),
                      "gamma": uniform(loc=0, scale=0.1),
                      "subsample": uniform(loc=0.5, scale=0.5),
                      "colsample_bytree": uniform(loc=0.5, scale=0.5),
                      "colsample_bylevel": uniform(loc=0.1, scale=0.9),
                      "reg_alpha": uniform(loc=0, scale=0.1),
                      "reg_lambda": uniform(loc=0.9, scale=0.1)}
clf_cv = RandomizedSearchCV(clf, param_distributions, n_iter=50, scoring="roc_auc", n_jobs=4,
                   refit=True, cv=cv, verbose=1, pre_dispatch='2*n_jobs', error_score='raise',
                   return_train_score='warn')

In [ ]:
clf_cv.fit(X, y)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


Get best model from random search cv

In [ ]:
best_clf = clf_cv.best_estimator_
print(clf_cv.best_score_)
print(clf_cv.best_params_)

# Assess final model on 20-fold stratified cv

In [ ]:
cv = StratifiedKFold(n_splits=20, shuffle=True)
y_pred = cross_val_predict(best_clf, X, y, cv=cv, n_jobs=3, method="predict_proba", verbose=2)
roc_auc_score(y, y_pred[:,1])

# Refit final model on all data

In [ ]:
clf.fit(X, y)

# Generate predictions on test set

In [ ]:
X_test = df_test.iloc[:,2:].values
id_test = df_test["Resident ID"]
X_test.shape

In [ ]:
y_pred_test = clf.predict_proba(X_test)[:,1]

In [ ]:
df_pred = pd.DataFrame(np.array([id_test, y_pred_test]).T, columns=["Resident ID", "Disease Flag"])
df_pred.head()

In [ ]:
df_pred.to_csv("submission.csv", index=False)